

# Phase2 Ablation

## Download files

In [ ]:
!wget https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/real-time-object-detection-deep-learning/real-time-object-detection.zip
!unzip -qq real-time-object-detection.zip
%cd real-time-object-detection

--2021-06-11 06:01:06--  https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/real-time-object-detection-deep-learning/real-time-object-detection.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.229.48
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.229.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21376463 (20M) [application/zip]
Saving to: ‘real-time-object-detection.zip.1’

real-time-object-de 100%[===================>]  20.39M  8.55MB/s    in 2.4s    

2021-06-11 06:01:09 (8.55 MB/s) - ‘real-time-object-detection.zip.1’ saved [21376463/21376463]

replace real-time-object-detection/MobileNetSSD_deploy.caffemodel? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
/content/real-time-object-detection


In [ ]:
from google.colab import files
uploaded = files.upload()

### Import Packages

In [ ]:
# import the necessary packages
from matplotlib import pyplot as plt
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import cv2

### Function to display images in Jupyter Notebooks and Google Colab

In [ ]:
def plt_imshow(title, image):
	# convert the image frame BGR to RGB color space and display it
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	plt.imshow(image)
	plt.title(title)
	plt.grid(False)
	plt.show()

### Object detection in video with deep learning and OpenCV

In [ ]:
# first, let's get a video on which we can run our face mask detector 
!wget https://colab-notebook-videos.s3-us-west-2.amazonaws.com/janie.mp4

--2021-06-11 06:01:22--  https://colab-notebook-videos.s3-us-west-2.amazonaws.com/janie.mp4
Resolving colab-notebook-videos.s3-us-west-2.amazonaws.com (colab-notebook-videos.s3-us-west-2.amazonaws.com)... 52.218.133.42
Connecting to colab-notebook-videos.s3-us-west-2.amazonaws.com (colab-notebook-videos.s3-us-west-2.amazonaws.com)|52.218.133.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3413203 (3.3M) [video/mp4]
Saving to: ‘janie.mp4.1’

janie.mp4.1         100%[===================>]   3.25M  2.88MB/s    in 1.1s    

2021-06-11 06:01:24 (2.88 MB/s) - ‘janie.mp4.1’ saved [3413203/3413203]



In [ ]:
# construct the argument parse and parse the arguments
#ap = argparse.ArgumentParser()
#ap.add_argument("-p", "--prototxt", required=True,
#	help="path to Caffe 'deploy' prototxt file")
#ap.add_argument("-m", "--model", required=True,
#	help="path to Caffe pre-trained model")
#ap.add_argument("-c", "--confidence", type=float, default=0.2,
#	help="minimum probability to filter weak detections")
#args = vars(ap.parse_args())
# since we are using Jupyter Notebooks we can replace our argument
# parsing code with *hard coded* arguments and values
args = {
	"input" : "videoP.mp4",
	"output" : "output.avi",
	"prototxt": "MobileNetSSD_deploy.prototxt.txt",
	"model": "MobileNetSSD_deploy.caffemodel",
	"confidence" : 0.5
}

In [ ]:
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

[INFO] loading model...


In [ ]:
# grab a reference to the video file, initialize pointer to output
# video file, and initialize the FPS counter
print("[INFO] opening video file...")
vs = cv2.VideoCapture(args["input"])
writer = None
fps = FPS().start()

[INFO] opening video file...


## Helpers

### Classes

In [ ]:
class rectangle:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
      
    def coordinates(self):
      return self.x1, self.y1, self.x2, self.y2

### Functions

In [ ]:
def overlap(r1, r2):
    #TODO: Function
    if (r1.x2>=r2.x1 and r1.x2 <=r2.x2 and r1.y2>=r2.y1 and r1.y2 <=r2.y2):
      return True
    if (r1.x1>=r2.x1 and r1.x1 <=r2.x2 and r1.y1>=r2.y1 and r1.y1 <=r2.y2):
      return True
    if (r1.x2>=r2.x1 and r1.x2 <=r2.x2 and r1.y1>=r2.y1 and r1.y1 <=r2.y2):
      return True
    if (r1.x1>=r2.x1 and r1.x1 <=r2.x2 and r1.y2>=r2.y1 and r1.y2 <=r2.y2):
      return True
    
    if (r2.x2>=r1.x1 and r2.x2 <=r1.x2 and r2.y2>=r1.y1 and r2.y2 <=r1.y2):
      return True
    if (r2.x1>=r1.x1 and r2.x1 <=r1.x2 and r2.y1>=r1.y1 and r2.y1 <=r1.y2):
      return True
    if (r2.x2>=r1.x1 and r2.x2 <=r1.x2 and r2.y1>=r1.y1 and r2.y1 <=r1.y2):
      return True
    if (r2.x1>=r1.x1 and r2.x1 <=r1.x2 and r2.y2>=r1.y1 and r2.y2 <=r1.y2):
      return True
    return False

def count_cases(rectangles):
    overlaped = 0
    for i in range(1,len(rectangles)):
        for j in range(1,len(rectangles)):
            if overlap(rectangles[0],rectangles[j]):
                overlaped +=1
                #rectangles[0].coordinates()
                #rectangles[j].coordinates()
                #print()
                break
        rectangles.pop(0)    

    return overlaped


## Predict video

In [ ]:
vs = cv2.VideoCapture(args["input"])

In [ ]:
total_cases = []
total_rectangles = []
# loop over the frames from the video stream
while True:
  
	# grab the next frame
	frame = vs.read()[1]

	# if we did not grab a frame then we have reached the end of the
	# video
	if frame is None:
		break
	
	# resize the frame to have a maximum width of 400 pixels
	frame = imutils.resize(frame, width=400)

	# grab the frame dimensions and convert it to a blob
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)

	# pass the blob through the network and obtain the detections and
	# predictions
	net.setInput(blob)
	detections = net.forward()
	rectangles = []
  

 	# loop over the detections
	for i in np.arange(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the prediction
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the `confidence` is
		# greater than the minimum confidence
		if confidence > args["confidence"]:
			# extract the index of the class label from the
			# `detections`, then compute the (x, y)-coordinates of
			# the bounding box for the object
			idx = int(detections[0, 0, i, 1])
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX,startY,endX,endY) = box.astype("int")
			rectangles.append(rectangle(startX,startY,endX,endY))
			# draw the prediction on the frame
			label = "{}: {:.2f}%".format(CLASSES[idx],
				confidence * 100)
			cv2.rectangle(frame,(startX, startY), (endX, endY), COLORS[idx], 2)
			y = startY - 15 if startY - 15 > 15 else startY + 15
			
		aux = []
		for i,j in enumerate(rectangles):
			print(i)
			print(j.coordinates())
			aux.append(j)

	total_rectangles.append(aux)
	overlaped = count_cases(rectangles)
	total_cases.append(overlaped)
	frame = cv2.putText(frame, 'Cases:'+str(overlaped), (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 1, cv2.LINE_AA)
	# update the FPS counter
	fps.update()

	# if the video writer is None *AND* we are supposed to write
	# the output video to disk initialize the writer
	if writer is None and args["output"] is not None:
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 20,
			(frame.shape[1], frame.shape[0]), True)
  
	# if the writer is not None, write the frame with recognized
	# faces to disk
	if writer is not None:
		writer.write(frame)
  
# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
vs.release()

# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

Streaming output truncated to the last 5000 lines.
0
(312, 9, 396, 223)
0
(312, 9, 396, 223)
0
(312, 9, 396, 223)
0
(312, 9, 396, 223)
0
(312, 9, 396, 223)
0
(312, 9, 396, 223)
0
(312, 9, 396, 223)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(323, 9, 398, 216)
0
(3

In [ ]:
for i in total_rectangles:
    for j,k in enumerate(i):
        print(j)
        print(k.coordinates())

In [ ]:
set(total_cases)

{0, 1, 2, 3}

In [ ]:
!ffmpeg -i output.avi output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

### Display video inline

In [ ]:
from IPython.display import HTML
from base64 import b64encode

mp4 = open("output.mp4", "rb").read()
dataURL = "data:videoP/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % dataURL)

For a detailed walkthrough of the concepts and code, be sure to refer to the full tutorial, [*Real-time object detection with deep learning and OpenCV*](https://www.pyimagesearch.com/2017/09/18/real-time-object-detection-with-deep-learning-and-opencv/)  published on 2017-09-18.


# Code License Agreement
```
Copyright (c) 2020 PyImageSearch.com

SIMPLE VERSION
Feel free to use this code for your own projects, whether they are
purely educational, for fun, or for profit. THE EXCEPTION BEING if
you are developing a course, book, or other educational product.
Under *NO CIRCUMSTANCE* may you use this code for your own paid
educational or self-promotional ventures without written consent
from Adrian Rosebrock and PyImageSearch.com.

LONGER, FORMAL VERSION
Permission is hereby granted, free of charge, to any person obtaining
a copy of this software and associated documentation files
(the "Software"), to deal in the Software without restriction,
including without limitation the rights to use, copy, modify, merge,
publish, distribute, sublicense, and/or sell copies of the Software,
and to permit persons to whom the Software is furnished to do so,
subject to the following conditions:
The above copyright notice and this permission notice shall be
included in all copies or substantial portions of the Software.
Notwithstanding the foregoing, you may not use, copy, modify, merge,
publish, distribute, sublicense, create a derivative work, and/or
sell copies of the Software in any work that is designed, intended,
or marketed for pedagogical or instructional purposes related to
programming, coding, application development, or information
technology. Permission for such use, copying, modification, and
merger, publication, distribution, sub-licensing, creation of
derivative works, or sale is expressly withheld.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND,
EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES
OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND
NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS
BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN
ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```